# Investigating the Network

## Intro / Goal

## Setup

## Device Mapping

| IP                                    | Sven's Device | Maxi's Device | Fabi's Devices                         |
| ------------------------------------- | ------------- | ------------- | -------------------------------------- |
| 192.168.178.1                         | Router        |               |                                        |
| 192.168.178.21                        |               | Android Phone |                                        |
| 192.168.178.24                        | Google Home   |               |                                        |
| 192.168.178.26                        | Chromecast    |               |                                        |
| 192.168.178.27                        | Google Home   |               |                                        |
| 192.168.178.29                        | Google Home   |               |                                        |
| 192.168.178.42                        | Google Home   |               |                                        |
| 192.168.178.43                        | SmartTV       |               |                                        |
| 192.168.178.44                        | Android Phone |               |                                        |
| 192.168.178.50                        | iPad          |               |                                        |
| 192.168.178.51                        | MacBook       |               |                                        |
| 192.168.178.58                        | Vacuum Robot  |               |                                        |
| 192.168.178.59                        |               | Notebook      |                                        |
| 192.168.178.60                        | MacBook       |               |                                        |
| 192.168.178.62                        | iPhone        |               |                                        |
| 192.168.178.64                        | SmartTV       |               |                                        |
| 2003:c1:3712:ac00:e9ad:724d:142a:c5c9 |               | Smartphone    |                                        |
| 2003:c1:3712:ac00:38ee:9c51:e7ee:fe52 |               | Notebook      |                                        |
| 192.168.0.1                           |               |               | Router                                 |
| 192.168.0.2                           |               |               | Wifi Smart Plug                        |
| 192.168.0.8                           |               |               | Amazon Fire TV                         |
| 192.168.0.9                           |               |               | Sonos Wifi Loudspeaker                 |
| 192.168.0.14                          |               |               | iPad                                   |
| 192.168.0.22                          |               |               | ESP32 Microcontroller                  |
| 192.168.0.88                          |               |               | Raspberry Pi used for network sniffing |
| 192.168.0.121                         |               |               | ESP32 with Feinstaubsensor firmware    |


In [ ]:
# mapping for later sections
#devices_labels = ['Router (Sven)', 'Google Home (Sven)', 'Chromecast (Sven)', 'Smart TV (Sven)', 'Android Phone (Sven)', 'iPad (Sven)', 'MacBook (Sven)', 'Vacuum Robot (Sven)', 'iPhone (Sven)',
#                  'Android Phone (Maxi)', 'Notebook (Maxi)',]
# no router traffic is analyzed due to potential issues caused by device <--> router communication
# '192.168.178.1': 'Router (Sven)',
devices_mapping = {
                    '192.168.178.21': 'Android Phone (Maxi)',
                    '192.168.178.24': 'Google Home (Sven)',
                    '192.168.178.26': 'Chromecast (Sven)',
                    '192.168.178.27': 'Google Home (Sven)',
                    '192.168.178.29': 'Google Home (Sven)',
                    '192.168.178.42': 'Google Home (Sven)',
                    '192.168.178.43': 'Smart TV (Sven)',
                    '192.168.178.44': 'Android Phone (Sven)',
                    '192.168.178.50': 'iPad (Sven)',
                    '192.168.178.51': 'MacBook (Sven)',
                    '192.168.178.58': 'Vacuum Robot (Sven)',
                    '192.168.178.59': 'Notebook (Maxi)',
                    '192.168.178.60': 'MacBook (Sven)',
                    '192.168.178.62': 'iPhone (Sven)',
                    '192.168.178.64': 'Smart TV (Sven)',
                    '2003:c1:3712:ac00:e9ad:724d:142a:c5c9': 'Android Phone (Maxi)',
                    '2003:c1:3712:ac00:38ee:9c51:e7ee:fe52': 'Notebook (Maxi)',
                    '192.168.0.2':    "Wifi Smart Plug (Fabi)",
                    '192.168.0.8':    "Amazon Fire TV (Fabi)",
                    '192.168.0.9':    "Sonos Wifi Loudspeaker (Fabi)",
                    '192.168.0.14':   "iPad (Fabi)",
                    '192.168.0.22':   "ESP32 Microcontroller (Fabi)",
                    '192.168.0.88':   "Raspberry Pi used for network sniffing (Fabi)",
                    '192.168.0.121':  "ESP32 with Feinstaubsensor firmware (Fabi)",
}

devices_labels = list(set(devices_mapping.values()))

Start of the program

In [ ]:
# Program, run imports
%matplotlib inline
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import time
import seaborn as sns
import requests
sns.set(style="darkgrid")

# used to skip merging and filtering
new_read = False

Create Pandas Dataframe

In [ ]:
if new_read:
    print('Concatenate csv files...')
    df_fabi = pd.read_csv("dumps/allPorts_fabian.csv", encoding = "latin")
    df_maxi = pd.read_csv("dumps/allPorts_maxi_v2.csv", encoding = "latin")
    df_sven = pd.read_csv("dumps/advanced-dumps-sven.csv", encoding = "latin")
    df = pd.concat([df_fabi, df_maxi, df_sven])
    print('Concatenate operation completed')
    print(len(df))

Filter Local Traffic

Print total number of captured packages

In [ ]:
if new_read:
    time_filter_start = time.time()
    print('Compute intensive task filter started...')
    # source and destination should not start with 192.168 to filter local network. DNS should not be filtered.
    filtered = df.loc[~df['Source'].str.startswith("192.168", na=False) & df['Destination'].str.startswith("192.168", na=False) |
                    df['Source'].str.startswith("192.168", na=False) & ~df['Destination'].str.startswith("192.168", na=False) |
                    df['Protocol'].isin(['DNS'])]

    filtered = filtered.loc[~filtered['Protocol'].isin(['DHCP', 'ARP', 'MDNS', 'LLDP', 'SSDP', 'IGMP', 'IGMPv2', 'IGMPv3', 'ICMP', 'ICMPv4', 'ICMPv6', 'ieee1905', 'LLMNR'])]
    filtered = filtered.loc[~filtered['Destination'].isin(['255.255.255.255'])]


    df = filtered 

    # sort array by timestamp

    df = df.sort_values(by=['Time'])
    # save to csv
    # df.to_csv('df.csv')
    # print (df.loc[df['Destination'].str.contains("255.255.255.255")])
    time_filter_duration = time.time() - time_filter_start
    print('Task filter completed in ', time_filter_duration, ' seconds')
else:
    df = pd.read_csv('df.csv')


add a new device coulmn 'Device Name' for better readability

In [ ]:
if not 'Device Name' in df.columns:
    new_read = True # save df later
    time_device_name_calc_start = time.time()
    print('Compute Device Name column...')
    #df_devices = pd.DataFrame({'Source': devices_mapping.keys(), 'Device Name': devices_mapping.values()})
    # df = df.merge(df_devices, on='Source', how='left')
    # Apply Device Name to incoming and outgoing packets
    df.loc[df['Source'].isin(devices_mapping.keys()), 'Device Name'] = df.loc[df['Source'].isin(devices_mapping.keys())].Source.apply(lambda x : devices_mapping[x])
    df.loc[df['Destination'].isin(devices_mapping.keys()), 'Device Name'] = df.loc[df['Destination'].isin(devices_mapping.keys())].Destination.apply(lambda x : devices_mapping[x])
    time_device_name_calc_duration = time.time() - time_device_name_calc_start
    print('Compute Device Name column completed in ', time_device_name_calc_duration, ' in seconds')
    print(df.head(10))

In [ ]:
# helper functions
def utcEntryToTimestamp(entry):
    #if '.' in entry:
    row_entry = entry.split(".")[0]
    #else:
    #    row_entry = entry.split(",")[0]
    TIME_FORMAT='%Y-%m-%d %H:%M:%S'
    ts = int(datetime.strptime(row_entry, TIME_FORMAT).timestamp())
    return ts

def utcRowToTimestamp(row):
    return utcEntryToTimestamp(row.at['Time'])
utcRowToTimestamp(df.iloc[0])
# print(df.loc[0].at['Time'])


In [ ]:
min_15_duration = 60 * 15
# add index time column, if it doesn't exist yet
if not 'index-time' in df.columns:
    time_utc_calc_start = time.time()
    print('Compute index-time column (approx. 5min)...')
    df['index-time'] = df['Time'].apply(lambda x: utcEntryToTimestamp(x)) # .apply(lambda x: x)) # map to 15 min window
    first_entry_offset = df.iloc[0].at['index-time'] // min_15_duration
    df['index-time'] = (df['index-time'] // min_15_duration) - first_entry_offset
    new_read = True
    time_utc_calc_duration = time.time() - time_utc_calc_start
    print('index-time column task completed in ', time_utc_calc_duration, ' seconds')

Save df to csv / load df from csv

In [ ]:
if new_read:
    df.to_csv('df.csv')

print(df)
print(len(df))

## Protocols
We would like to start our investigations by getting a better insight about the Packet Types were are sending to the web.
Several protocols exist for sending data trough the web nowadays. Based on their purpose, these cane be assigned to Application, Transport and Internet layer.
- Application layer
  - DNS
  - SSH
  - HTTP
  - HTTPS
  - ...
- Transport Layer
  - TCP
  - UDP
  - QUICK
- Internet Layer
  - IP (IPv4, IPv6)
  - ICMP
  - ...   

Some of these protocols, such as HTTP, UDP and TCP are quite old. Others, such as the QUICK protocol, were only introduced in the recent years. 



### Protocol Distribution among Frames

To get a better overview about all the traffic our devices use the sent and receive data from the Internet, we print out a ranking and plot the data. \
We see that the mayority of our captured frames use **TCP*** as underlying protocol. This is not surprising, since TCP is the most used transport protocol and is almost used for every traffic on the web. \
Surprisingly, we also see **QUICK** in our top 5. QUICK, which was recently standardized in May 2021, seems to be heavily pushed by its creator Google. As you can see from our device list, Google devices claim a large share out of all devices. This might be the reason for its popularity among our captured frames.

In [ ]:
# rank by most used protocols
df_ranked_protocols = df.groupby('Protocol').size()
# print(df_ranked_protocols.nlargest(15))
print(df_ranked_protocols)

# plot pie diagram
fig, ax = plt.subplots(figsize=(10,10))
plt.title('Protocol Distribution by Amount of Packets')
ax.pie(df_ranked_protocols, labels=df_ranked_protocols.keys(), autopct='%1.1f%%')
fig.show()

### Amount of Data Traffic per Protocol
We also had a look in the amount of data traffic that is sent by each protocol. It seemes that this heavily correlates with the protocol distribution among our captured frames. 

In [ ]:
df_data_per_protocol = df.groupby('Protocol')['Length'].sum()
print(df_data_per_protocol.nlargest(15))

fig, ax = plt.subplots(figsize=(10,10))
plt.title('Protocol Distribution by Data Traffic')
ax.pie(df_data_per_protocol, labels=df_data_per_protocol.keys(), autopct='%1.1f%%',)
fig.show()

### Average Data length per Protocol Type
In order to investigate which kind if protocol holds the most data, we compute the mean of each frame per protocol.
It seems that especially SSL and SSH have the greatest amount of data per frame. \
Surprisingly, we also some various industrial protocols, such as H1, PKIX-CRL and HART_IP in this ranking. These protocols are especially designed for sending large data packets. We are not very sure what programs on Maxi's laptop and phone are responsible for this and will investigate this further in a later section of this write-up.

In [ ]:
df_mean_protocol_packet_length = df.groupby('Protocol')['Length'].mean()
print(df_mean_protocol_packet_length.nlargest(15))
#print (df.loc[df['Protocol'].str.contains("HART_IP")])

# plot
fig, ax = plt.subplots()
ax.tick_params(axis='x', which='major', labelsize=12)
ax.tick_params(axis='x', which='minor', labelsize=12)
plt.xlabel('Protocol')
plt.ylabel('Data [Byte]')
plt.title('Mean Data Length per Protocol Type')
ax.bar(df_mean_protocol_packet_length.keys(), df_mean_protocol_packet_length, align='center',)
plt.xticks(rotation=60, ha="right")
fig.show()

## Source addresses

In [ ]:
df_ranked_sources = df.groupby('Source').size()
print(df_ranked_sources.nlargest(25))

Local devices with most traffic sent

In [ ]:
print(df.loc[df['Source'].str.startswith("192.168") & df['Source'].isin(devices_mapping.keys())].groupby(['Source', 'Device Name']).size())

## Destination Adresses

In [ ]:
df_ranked_destinations = df.groupby('Destination').size()
print(df_ranked_destinations.nlargest(25))

In [ ]:
# write ip address destinations to file
unique_dests = df['Destination'].unique()
file1 = open("destinations.txt","w")
for row in unique_dests:
    file1.write(row + "\n")
file1.close()

Local devices with most traffic received

In [ ]:
print(df.loc[df['Destination'].str.startswith("192.168")].groupby('Destination').size())

### IP Locations World Wide Sources (Incoming Traffic) ###

In [ ]:
rows = []
for address, count in df_ranked_sources.nlargest(30).iteritems(): # 30 entries limit 
    if address.startswith("192.168."):
        continue
    headers = { 'User-Agent': "keycdn-tools:https://www.example.com" }
    url = "https://tools.keycdn.com/geo.json?host={}".format(address)
    json_response = requests.get(url, headers=headers).json()
    # print(json_response)
    geo = json_response['data']['geo']
    # print(json_response)
    # print(geo)
    # geo['country_name']
    rows.append([geo['ip'], geo['country_code'], geo['longitude'], geo['latitude'], geo['isp']])
    
# as dataframe
df_coord = pd.DataFrame(rows, columns=["ip", "country", "lng", "lat", "isp"])
print(df_coord)
df_coord.to_csv('ipLocations.csv')

# plot on world
g_world = gpd.GeoDataFrame(df_coord, geometry=gpd.points_from_xy(df_coord.lng, df_coord.lat))

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
base = world.plot(color='white', edgecolor='black')
g_world.plot(ax=base, marker='o', color='red', markersize=5)
plt.show()

### IP Locations World Wide Destinations (Outgoing Traffic) ###

In [ ]:
rows = []
for address, count in df_ranked_destinations.nlargest(30).iteritems(): # 30 entries limit 
    if address.startswith("192.168."):
        continue
    headers = { 'User-Agent': "keycdn-tools:https://www.example.com" }
    url = "https://tools.keycdn.com/geo.json?host={}".format(address)
    json_response = requests.get(url, headers=headers).json()
    #print(json_response)
    geo = json_response['data']['geo']
    #print(json_response)
    #print(geo)
    rows.append([geo['ip'], geo['country_code'], geo['longitude'], geo['latitude'], geo['isp']])
    
# as dataframe
df_coord = pd.DataFrame(rows, columns=["ip", "country", "lng", "lat", "isp"])
print(df_coord)
df_coord.to_csv('ipLocations.csv')

# plot on world
g_world = gpd.GeoDataFrame(df_coord, geometry=gpd.points_from_xy(df_coord.lng, df_coord.lat))

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
base = world.plot(color='white', edgecolor='black')
g_world.plot(ax=base, marker='o', color='red', markersize=5)
plt.show()

## Encryption

To look at the percentage of encrypted traffic we will base our analysis solely on the used protocols. The reason behind this is that we were not able to capture the content of the traffic over time due to the memory intensity and therefore cannot look if the content is encrypted or not.

To distinguish between packages with encrypt content and packages with non-encrypt content we will only look at the following application layer protocols and divide them between non-encrypt and encrypted:
- Encrypted:
  - DTLSv1.2
  - ISAKMP
  - RTCP
  - SSH
  - SSL
  - TLS
  - WireGuard

- Non-Encrypted:
  - DNS
  - HTTP
  - MP4
  - NTP
  - OCSP
  - SMTP
  
- Uncategorized:
  - CLASSIC-STUN
  - HART_IP
  - HCrt
  - IMAP
  - STUN
  - XMPP

Based on this we get the following encryption distribution:

In [ ]:
filter_protocols = ['ESP', 'GQUIC', 'H1', 'LWAPP', 'MPTCP', 'QUIC', 'TCP', 'THRIFT', 'TURN CHANNEL', 'UDP', 'UDPENCAP']
encrypted_application_protocols = ['DTLSv1.2', 'ISAKMP', 'RTCP', 'SSHv2', 'SSH', 'SSL', 'SSLv2', 'SSLv3', 'TLSv1', 'TLSv1.2', 'TLSv1.3', 'WireGuard']
non_encrypted_application_protocols = ['DNS', 'HTTP', 'HTTP/JSON', 'HTTP/JSON/XML', 'HTTP/XML', 'MP4', 'NTP', 'OCSP', 'SMTP']
uncategorized_application_protocols = ['CLASSIC-STUN', 'HART_IP', 'HCrt', 'IMAP', 'STUN', 'XMPP/XML']

only_application_protocols = df.loc[~df['Protocol'].isin(filter_protocols)]
only_encrypted_protocols = only_application_protocols.loc[only_application_protocols['Protocol'].isin(encrypted_application_protocols)]
only_non_encrypted_protocols = only_application_protocols.loc[only_application_protocols['Protocol'].isin(non_encrypted_application_protocols)]
only_uncategorized_protocols = only_application_protocols.loc[only_application_protocols['Protocol'].isin(uncategorized_application_protocols)]

content_length_encrypted = only_encrypted_protocols['Length'].sum()
content_length_unencrypted = only_non_encrypted_protocols['Length'].sum()
content_length_uncategorized = only_uncategorized_protocols['Length'].sum()

labels = 'Encrypted', 'Non-Encrypted', 'Uncategorized'
fig, ax = plt.subplots(figsize=(10,10))
plt.title('Encryption Distribution')
ax.pie([content_length_encrypted, content_length_unencrypted, content_length_uncategorized], explode=(0.1, 0, 0), labels=labels, autopct='%1.1f%%',)
fig.show()

By inspecting the plot it can quickly be seen that the encrypted packages only take up a small portion of the traffic. However, this number can not be taken as an exact value but as an lower bound since packages send with the UDP and TCP protocol take up most of the traffic but could be encrypted as well. Since don't have the content of the packages we cannot further investigate into the encryption of these. But we suspect that the ecryption percentage would be a lot higher.

## IPv6 vs IPv4

In [ ]:
ether_distribution = df.groupby('EtherType').size()

fig, ax = plt.subplots()
plt.title('EtherType Distribution')
ax.pie(ether_distribution, labels=ether_distribution.keys(), autopct='%1.1f%%',)
fig.show()

### Which devices are using IPv6?

In [ ]:
ipv6_traffic = df.loc[df['EtherType'].isin(['IPv6'])]

print(ipv6_traffic.groupby(['Source']).size().nlargest(15))
# print(ipv6_traffic.groupby(['Source', 'Device Name']).size())

# TODO find out device names for rest of IP's

## DNS 

We would like to investigate further our collected data. Now, we are particular interested in DNS requests. therefore, we filter the data by the protocol "DNS".

In [ ]:
df_dns = df[df['Protocol'] == 'DNS']
print(df_dns.head(1))

### Used DNS Server

Various DNS Resolvers exist on the Internet. We would like to find out which our devices are using during their operation. 
Some devices can be configured to use specif DNS Servers, some a DNS resolver hardcoded in their firmware. In man cases the resolve request is just forwarded to the router, who takes care of this. 
By grouping our DNS Destinations and counting the requests, we see which resolvers were primarily used.

In [ ]:
df_dns_server = df_dns.loc[~df_dns['Info'].str.contains("response")]
print(df_dns_server.groupby(['Destination']).size())

We see that most of our DNS requests were send to our routers or well known DNS resolvers, such as Google's 8.8.8.8. However, we also found tree unknown DNS resolvers. A print out of these requests reveals that the Feinstaubsensor IoT device is talking to changing DNS Servers to resolve its destinations.
The other one seems to be an IPv6 DNS resolver, that is used by our Amazon fire TV to resolve its requests. 

In [ ]:
print(df_dns_server.loc[df_dns_server['Destination'].str.contains("195.234.128.139")].head(5))
print(df_dns_server.loc[df_dns_server['Destination'].str.contains("217.68.162.126")].head(5))
print(df_dns_server.loc[df_dns_server['Destination'].str.contains("2a02:2457:10c:101::126")].head(5))

Another great insight is figuring out what URLs are actually being resolved. This can also be used to track a devices browsing behavior. We count the requests by URL to get a better overview. \
An interesting finding is that many of our top resolved URLs are requested by static devices, such as the Feinstaubsensor, Google Home or the Amazon Fire TV stick. 

Especially on personal computers and mobile devices, a browsing behavior from DNS requests can be found. This includes our love for Reddit. 👨🏽‍🚀

In [ ]:
# only look at standart query dns requests
df_dns_requests = df_dns[df_dns['Info'].str.contains('Standard query')]
# exclude responses from count
df_dns_requests = df_dns_requests[~df_dns_requests['Info'].str.contains('response')]
# display URLS
df_dns_urls = df_dns_requests['Info'].apply(lambda x: x.split(' ')[-1]).value_counts()
print('ns-287.awsdns-35.com' in df_dns_urls.keys())
print(df_dns_urls.head(30))

### Data traffic over time ###

For following sections, we form 15 minute timeframes and assign data traffic to those timeframes.

In [ ]:
def timeMapping(x):
    # apply time difference UTC+1 to labels
    time = datetime.utcfromtimestamp(x*min_15_duration + 3600)
    if time.minute == 0 and time.hour % 6 == 0:
        return time.strftime("%H:%M")
    else:
        return ""

In [ ]:




first_entry = df.iloc[0].at['index-time']
last_entry = df.iloc[-1].at['index-time']

print('from ', df.iloc[0].at['Time'], " to ", df.iloc[-1].at['Time'])

x_values_packets = list(range(first_entry, last_entry + 1))
print('Amount of Intervals',len(x_values_packets))

mapping_res = list(map(timeMapping, x_values_packets))


In [ ]:
# flag for third computation task
values_calculated = False

count_packets = np.zeros(last_entry - first_entry + 1)
length_packets = np.zeros(last_entry - first_entry + 1)

df_dict = None


In [ ]:
if not values_calculated:
    time_start = time.time()
    print('Compute packet count and data traffic over time 2/2...')
    df_traffic_grouped = df.groupby(['index-time']).Length.sum()
    df_packet_grouped = df.groupby(['index-time']).count()

    for timeframe, values in df_packet_grouped.iterrows():
        # print(ip, timeframe, values['Length'])
        # calculate index
        count_packets[timeframe] += values['Length']
    # break
    for timeframe, values in df_traffic_grouped.iteritems():
        length_packets[timeframe] += values
    length_packets = length_packets // 1000
    time_end = time.time()
    print(time_end - time_start)

# Version 1: aggregate packet length and data traffic
if not values_calculated:
    values_calculated = True
    print('Compute intensive task 3/3...')
    print('Task takes approximately 5 to 10 minutes.')
    time_start = time.time()
    if df_dict is None:
        print("Convert dataframe to dictionary structure")
        df_dict = df.to_dict('records')
        print("Conversion completed in ", time.time() - time_start, "seconds")
    for row in df_dict:
        count_packets[row['index-time']] += 1
        length_packets[row['index-time']] += row['Length']

    # old solution:
    #for index, row in df.iterrows():
    #    count_packets[row.at['index-time']] += 1
    #    length_packets[row.at['index-time']] += row.at['Length']

    time_end = time.time()
    print(time_end - time_start)

    length_packets = length_packets // 1000




In [ ]:
# print(count_packets)
# print(length_packets)
plt.rcParams['figure.figsize'] = [20, 5]
fig, ax = plt.subplots()
plt.xlabel('Time')
plt.ylabel('Packets')
plt.title('Packets per timeframe (15 min interval)')
plt.xticks(x_values_packets, mapping_res)
ax.bar(x_values_packets, count_packets, color='black')
fig.show()

fig, ax = plt.subplots()
plt.rcParams['figure.figsize'] = [20, 5]
plt.xlabel('Time')
plt.ylabel('Data [kB]')
plt.title('Data traffic per timeframe (15 min interval)')
plt.xticks(x_values_packets, mapping_res)
ax.bar(x_values_packets, length_packets, color='black')
fig.show()

## Device Evaluation ##

In [ ]:
# define data structure for device metadata
df_device_traffic  = pd.DataFrame() # counts incoming and outgoing data
df_device_packets = pd.DataFrame() # counts incoming and outgoing packets
df_device_dns = pd.DataFrame() # counts dns requests per device

# initialize data structure
for name in devices_labels:
    count_packets_device = np.zeros(2 * (last_entry - first_entry + 1)) # track incoming and outgoing traffic
    length_packets_device = np.zeros(2 * (last_entry - first_entry + 1))
    dns_count = np.zeros(df_dns_urls.size)
    df_device_packets[name] = count_packets_device
    df_device_traffic[name] = length_packets_device
    df_device_dns[name] = dns_count

In [ ]:
# Version 2
# print(devices_mapping.values())
# incoming traffic
run_device_analysis = True

if run_device_analysis:
    print('Process incoming and outgoing traffic per device...')
    # Outgoing traffic: Device sends traffic
    df_device_out = df.loc[(df['Source'].isin(devices_mapping.keys()))]
    df_device_out_packets = df_device_out.groupby(['Source', 'index-time']).count()
    df_device_out_data = df_device_out.groupby(['Source', 'index-time']).Length.sum()
    # Incoming traffic: Device receives packet
    df_device_in = df.loc[(df['Destination'].isin(devices_mapping.keys()))]
    df_device_in_packets = df_device_in.groupby(['Destination', 'index-time']).count()
    df_device_in_data = df_device_in.groupby(['Destination', 'index-time']).Length.sum()

    # filter DNS request packets, device must have send request
    df_dns_frames = df.loc[df['Source'].isin(devices_mapping.keys()) & (df['Protocol'] == 'DNS') & (df['Info'].str.contains('Standard query'))]
    df_dns_frames = df_dns_frames.loc[~df_dns_frames['Info'].str.contains('response')]
    # apply map function on all frames
    print('Compute dns request apply function...')
    df_dns_frames['dns-request'] = df_dns_frames['Info'].apply(lambda x: x.split(" ")[-1])

    # group by device
    df_dns_device = df_dns_frames.groupby(['Device Name', 'dns-request']).Length.count().sort_values(ascending=False)
# print(df_dns_device)

# ip address traffic
df_device_ip_dest = df.loc[(df['Source'].isin(devices_mapping.keys()))]
df_device_ip_dest = df_device_ip_dest.groupby(['Device Name', 'Destination']).Length.count().sort_values(ascending=False)
df_device_ip_dest.head(50)

In [ ]:
# transfer processed data to data structures
# print(df_device_out_packets)
# Process Outgoing traffic
for (ip, timeframe), values in df_device_out_packets.iterrows():
    # print(ip, timeframe, values['Length'])
    # calculate index
    index = 2 * timeframe + 1
    df_device_packets.loc[index, devices_mapping[ip]] += values['Length']
    # break
for (ip, timeframe), values in df_device_out_data.iteritems():
    index = 2 * timeframe + 1
    df_device_traffic.loc[index, devices_mapping[ip]] += values

# Process Incoming traffic
for (ip, timeframe), values in df_device_in_packets.iterrows():
    # print(ip, timeframe, values['Length'])
    # calculate index
    index = 2 * timeframe
    df_device_packets.loc[index, devices_mapping[ip]] += values['Length']
    # break
for (ip, timeframe), values in df_device_in_data.iteritems():
    index = 2 * timeframe
    df_device_traffic.loc[index, devices_mapping[ip]] += values

# Process DNS requests
dns_keys = list(df_dns_urls.keys())
for (label, address), values in df_dns_device.iteritems():
    index = dns_keys.index(address)
    df_device_dns.loc[index, label] += values
# print(df_dns_frames, type(df_dns_frames))

In [ ]:
# save to csv
df_device_traffic.to_csv('df_device_traffic.csv')
df_device_packets.to_csv('df_device_packets.csv')
df_device_dns.to_csv('df_device_dns.csv')

### Traffic by Device ###
- Amount traffic per device (bar chart)
- Amount packets per device (bar chart)

In [ ]:
# for each device, plot a bar that shows the data traffic (incoming and outgoing) and amount of packets
# aggregate data
df_device_in_total = (df_device_traffic.iloc[::2].agg(['sum']) // 1000).transpose()# even rows, incoming traffic
df_device_out_total = (df_device_traffic.iloc[1::2].agg(['sum']) //1000).transpose()# odd rows, outgoing traffic
df_device_in_total_num = df_device_in_total['sum'].to_numpy()
df_device_out_total_num = df_device_out_total['sum'].to_numpy()

df_device_in_packets_total = (df_device_packets.iloc[::2].agg(['sum'])).transpose()# even rows, incoming traffic
df_device_out_packets_total = (df_device_packets.iloc[1::2].agg(['sum'])).transpose()# odd rows, outgoing traffic
df_device_in_packets_total_num = df_device_in_packets_total['sum'].to_numpy()
df_device_out_packets_total_num = df_device_out_packets_total['sum'].to_numpy()


fig, ax = plt.subplots()
plt.rcParams['figure.figsize'] = [20, 5]
plt.xlabel('Devices')
plt.ylabel('Data [kB]')
plt.title('Data traffic per Device')
plot_x_devices_traffic = np.arange(len(devices_labels))
plt.xticks(plot_x_devices_traffic, devices_labels, rotation=60, ha="right")
ax.bar(plot_x_devices_traffic, df_device_in_total_num, width=0.4, color='green')
ax.bar(plot_x_devices_traffic + 0.4, df_device_out_total_num, width=0.4, color='red')
fig.show()

# plot packets

fig, ax = plt.subplots()
plt.rcParams['figure.figsize'] = [20, 5]
plt.xlabel('Devices')
plt.ylabel('Packets')
plt.title('Packets per Device')
plot_x_devices_traffic = np.arange(len(devices_labels))
plt.xticks(plot_x_devices_traffic, devices_labels, rotation=60, ha="right")
ax.bar(plot_x_devices_traffic, df_device_in_packets_total_num, width=0.4, color='green')
ax.bar(plot_x_devices_traffic + 0.4, df_device_out_packets_total_num, width=0.4, color='red')
fig.show()

### Device Activity ###
- Device activity per timeframe (plot chart) (1/0 plot)

The following plot shows at which timeframes packets from or to the device were captured. A single captured packet in a timeframe is sufficient to identify the device as "active". It does not show user activity, but it visualizes when the device was connected to the network and if it was used for data exchange with the web.


In [ ]:
# plot activity
# activity = 1 packet send/received within a timeframe
packets_1 = df_device_packets.iloc[::2].to_numpy()
packets_2 = df_device_packets.iloc[1::2].to_numpy()

combined = np.minimum(np.add(packets_1, packets_2), 1).transpose()
# print(df_device_packets.iloc[::2])
print(combined.shape)
plot_X_activity = np.arange(combined.shape[1])

# print(np.minimum(combined[0], (len(combined[0])) * [1] ))

# df_device_packets.iloc[1::2]
fig, ax = plt.subplots(nrows=combined.shape[0], ncols=1, figsize=[20, 40])
# plt.figure(figsize=[20, 30])
# plt.rcParams['figure.figsize'] = [20, 40]
#plt.xlabel('Time')
#plt.ylabel('Activity')
#plt.title('Packets per Device')
# plt.xticks(x_values_packets, mapping_res)
plt.setp(ax, xticks=x_values_packets, xticklabels=mapping_res,
        yticks=[0, 1],xlabel='Time', ylabel="Activity", )

for index in range(combined.shape[0]):
    # plt.subplot(combined.shape[0], 1, index+1)
    if index % 2 == 0:
        ax[index].plot(x_values_packets, combined[index], label=devices_labels[index], color="red")
    else:
        ax[index].plot(x_values_packets, combined[index], label=devices_labels[index], color="blue")
    ax[index].legend()


fig.show()

### Night Behaviour and Application Behaviour ###
- Top DNS request targets day/night (6 hour intervals)
- Analyze night behaviour (0-6 am) (low activity, find destination IP addresses, country, isp, protocols)
- Outgoing/Incoming traffic difference
- Application Activity (e.g. Google, Twitch, MVG, etc.)

We wanted to know how devices behaved at night without user interaction. Therefore, we decided to only use captured data between 0 and 6am for the first section.
In the second section, we want to find out more about application behaviour, e.g. when any Google application is active. Therefore, we assign to each larger application a set of ip addresses (e.g. via resolving collected dns requests) and plot the activity of the applications.

# Version 1 Slow Device Traffic Analysis, Duration approx. 6400 seconds
print(devices_mapping)
dns_keys = list(df_dns_urls.keys())
print('Start Comoute intensive device task...')
print('Takes approximately 30 min.')
time_dev_start = time.time()
for ip, label in devices_mapping.items():
    print(ip, label)
    df_dev = df.loc[(df['Source'] == ip) | (df['Destination'] == ip)]
    df_inc = df_dev.loc[df_dev['Destination'] == ip]

    df_out = df_dev.loc[df_dev['Source'] == ip]
    df_dns_device = df_out.loc[(df_out['Protocol'] == 'DNS') & df_out['Info'].str.contains('Standard query') & ~df_out['Info'].str.contains('response')]
    # process incoming traffic
    for inc_frame in df_inc.to_dict('records'):
        df_device_packets.loc[inc_frame['index-time']* 2, label] += 1
        df_device_traffic.loc[inc_frame['index-time']* 2, label] += inc_frame['Length']
    # process dns
    for dns_frame in df_dns_device.to_dict('records'):
        name = dns_frame['Info'].split(' ')[-1]
        # print(name, dns_keys.index(name))
        # add to device specific table
        df_device_dns.loc[dns_keys.index(name), label] += 1
    # process outgoing traffic
    for out_frame in df_out.to_dict('records'):
        df_device_packets.loc[out_frame['index-time']* 2 + 1, label] += 1
        df_device_traffic.loc[out_frame['index-time']* 2 + 1, label] += out_frame['Length']
#print(df_dev)
print("Task completed in ", time.time() - time_dev_start, " seconds")

Find the mule
Suspicious H1 protocol sends large amount of data to cloud
MVG thingy
Staubsauger

End of the notebook